# Example showing how to compute the offset calibration using the cross-talk effect in LinoSPAD2.

TODO WRITE TEXT
Import functions and set absolute path to .dat files


In [1]:
import matplotlib.pyplot as plt
from daplis.functions import cross_talk, fits
path = r"C:\Users\Ondra\Desktop\dezertacka\data\offset_w_LED"

Now, calculate timestamp differences for the pixels of interest. The result is saved in the 'cross_talk_data' folder at the same location where '.dat' data files are located.

In [ ]:
#Dont forget to set every parameter correctly here based on your data!
cross_talk.cross_talk_offset_collect(
    path,
    rewrite=True,
    daughterboard_number="B7d",
    motherboard_number="#28",
    firmware_version="2212s",
    timestamps=100,
)



> > > Collecting cross-talk data < < <

Creating pixel pairs from 0 to 255


Calculating timestamp differences between all pixels

! ! ! Feather file with timestamps differences already exists and will be rewritten ! ! !


! ! ! Deleting the file in 5 ! ! !


! ! ! Deleting the file in 4 ! ! !


! ! ! Deleting the file in 3 ! ! !


! ! ! Deleting the file in 2 ! ! !


! ! ! Deleting the file in 1 ! ! !




> > > Timestamp differences are saved as0000072934-0000073033_pixels_130-131.feather in C:\Users\Ondra\Desktop\dezertacka\data\offset_w_LED\cross_talk_data < < <


After collecting the timestamp differences, we can plot the CT peaks and offset them to zero and visualise them.

In [36]:
#TODO why after 130,131 error?
correct_pix_address = False
print("Creating pixel pairs from 0 to 255")
pix_pairs = [[i, i+1] for i in range(255)]
if correct_pix_address:
    for pi, pixels in enumerate(pix_pairs):
        for i, pixel in enumerate(pixels):
            if pixel > 127:
                pixels[i] = 255 - pixel
            else:
                pixels[i] = pixel + 128
        pix_pairs[pi] = pixels
# Collecting sensor population
mus = []
multiplier = 10
for pixels in pix_pairs:
    # if pixels == [130,131]:
    #     break
    mu, sigma = cross_talk.cross_talk_offset_plot(
    path,
    pixels,
    show_plot=False, multiplier = multiplier
    )
    mus.append(mu)
# print([float(mu) for mu in mus])
for pixels in pix_pairs:
    # if pixels == [130,131]:
    #     break
    cross_talk.plot_overlayed_data(path, pixels,show_plot=False, multiplier = multiplier)
plt.show()
for idx, pixels in enumerate(pix_pairs):
    # if pixels == [130,131]:
    #     break
    cross_talk.plot_normalized_data(path, pixels, mus[idx], multiplier = multiplier)
plt.show()



Creating pixel pairs from 0 to 255


ValueError: No valid data points found within the specified window.

# Solving the Offset Equations using a Linear Matrix Equation

## Formulation of the System

We have the system of equations:

```math
(t_i + d_i) - (t_4 + d_4) = d_i - d_4 = 	ext{off}_{i4}, \quad i \in [1,3].
```

```math
(t_0 + d_0) - (t_j + d_j) = d_0 - d_j = 	ext{off}_{0j}, \quad j \in [4,255].
```

```math
\sum_{k=1}^{255} 	ext{off}_{0k} = 0.
```

Rewriting this in matrix form:

```math
Ax = b
```

where:
- \( A \) is a bidiagonal matrix,
- \( x \) represents the unknown offset values,
- \( b \) is the vector of given offsets.

## Bidiagonal Matrix Structure

The system can be represented in matrix form as:
\[
\begin{bmatrix}
1 & -1 & 0 & 0 & \cdots & 0 \\
0 & 1 & -1 & 0 & \cdots & 0 \\
0 & 0 & 1 & -1 & \cdots & 0 \\
\vdots & \vdots & \vdots & \vdots & \ddots & \vdots \\
0 & 0 & 0 & 0 & \cdots & -1
\end{bmatrix}
\begin{bmatrix}
d_1 \\
d_2 \\
d_3 \\
\vdots \\
d_{255}
\end{bmatrix}
=
\begin{bmatrix}
\text{off}_{14} \\
\text{off}_{24} \\
\text{off}_{34} \\
\vdots \\
\text{off}_{0255}
\end{bmatrix}
\]

```

## Solving the System in Python

Since \( A \) is a bidiagonal matrix, we can solve it efficiently using forward and backward substitution. Below is a Python implementation:


In [ ]:
import numpy as np
import copy
offsets = copy.copy(mus)

#append 0 to mus which will be for the sum = 0
offsets.append(0)
A0 = np.eye(len(offsets))
A1 = -1*np.eye(len(offsets),k=1)
A = A0 + A1
A[-1,:] = 1
print(A.shape, len(offsets))

#TODO add condition on d_i > 0?
d = np.linalg.solve(A,offsets)
print(d)

(131, 131) 131
[-4.42647120e+02  3.84555791e+03 -3.93375424e+03  6.10913409e+02
 -2.98632540e+03 -3.81462502e+03  1.26231185e+03 -1.50853634e+03
 -3.39532006e+03  8.74153165e+02 -5.26032642e+03 -3.01712921e+03
  1.23067168e+03 -2.16271675e+03 -5.19405512e+03 -2.27392969e+03
 -5.00003850e+03 -3.17456517e+03  1.65441122e+03 -3.07279218e+03
  3.08439133e+03 -1.00886420e+03  3.56914051e+03 -6.76479057e+03
 -2.13854492e+03 -6.86163970e+03  1.71421718e+03 -1.40192939e+03
 -3.82681054e+02  1.44939991e+03  2.82534641e+03  1.57611674e+03
  1.37180691e+03  1.89138836e+03  5.43433436e+02 -1.11698872e+03
 -1.39320259e+00 -2.76664859e+03  7.83578053e+02  9.10798427e+02
  3.00043711e+03 -2.39038033e+03  6.84146988e+02 -8.16783945e+01
  1.81224518e+03  3.86937049e+02 -1.09086211e+02 -9.12328778e+02
  2.11826974e+03 -8.02205530e+01 -9.98909388e+02  2.19942781e+03
 -4.09186883e+03  5.60817008e+02  8.47920189e+02  3.50326429e+03
 -4.85489519e+03 -3.07140095e+03  2.30457438e+03  1.42800366e+03
  2.471598